# Welcome to **MolSearch !!** 


**💼 Initialization of the Notebook** <br> 

In [ ]:
welcome to this notebook

# ChemCluster Project Report  
**CH-200 – Practical Programming in Chemistry**  
**Group:** Romain Guichonnet, Elisa Rubbia, Flavia Zabala.
**Date:** 26 May 2025  

## 1. Introduction

*TODO: Write the project motivation, chemical context, and objectives here.*


## 2. Installation and Quickstart

```bash
# TODO: Clone repo, create env, install deps, launch app
git clone https://github.com/erubbia/ChemCluster.git
cd ChemCluster
conda env create -f environment.yml
conda activate chemcluster-env
streamlit run app.py


## 3. Project Structure

The repository is organized as follows:

